<a href="https://colab.research.google.com/github/AamnaKamran/Aether/blob/main/Update_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rdflib
!pip install SPARQLWrapper
!pip install owlready2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260905 sha256=b6a546e2f7e407704b65d959d9bcb89ff7a38085536704adb0d0965464c34559
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [14]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import FOAF, XMLNS, XSD, RDF, RDFS, OWL
import os
from google.colab import drive
drive.mount('/gdrive')
os.chdir('/content/')

# find if directory is '../Megascans Library/Downloaded/UAssets' or '../Megascans Library/UAssets'
def trigger_update(path_to_asset_folder): 
  
  
  g = Graph()
  g.parse('Populated_Assets_KG.ttl', format = "ttl")
  n = Namespace("http://www.semanticweb.org/szm/megascan-assets-ontology#")


  qres = g.query(
    """
      PREFIX mega:<http://www.semanticweb.org/szm/megascan-assets-ontology#>
      SELECT ?s ?aID ?aName WHERE {
      ?s mega:assetID ?aID.
      ?s mega:assetName ?aName.
       FILTER BOUND(?aID)
    }""")
  

  cur_KG_assets = set()    
  for row in qres:
    # check asset IDs currently loaded in the assets' KG - they have IDs same as their folder and json file names
    cur_KG_assets.add(row.aID.toPython())
  cur_KG_assets = list(cur_KG_assets)

  # cur_dir = os.chdir(path_to_asset_folder)
  # Get list of all asset folder names, then iteratively go through them.
  
  
  downloaded_asset_folder_names = os.listdir(path_to_asset_folder)
  downloaded_assets_path_dict = {}
  for folder_name in downloaded_asset_folder_names:
    if len(os.listdir(str(path_to_asset_folder)+str(folder_name)+'/')) != 0 and str(folder_name)+'.json' in os.listdir(str(path_to_asset_folder)+str(folder_name)+'/'):
      downloaded_assets_path_dict[folder_name] = str(path_to_asset_folder)+str(folder_name)+'/'+str(folder_name)+'.json'


# DONE  ########  - test if a new asset is correctly recognized (that's downloaded, but not added to KG)
# DOING ########  - then add only new assets to KG 

  print(downloaded_assets_path_dict)

  new_asset_paths_dict = {}
  for key,value in downloaded_assets_path_dict.items():
    if key not in cur_KG_assets:
      new_asset_paths_dict[key] = value
  print("Assets not yet added to KG:", new_asset_paths_dict)

trigger_update( '/content/Megascans Library/Downloaded/UAssets/')





Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
{'tlesfb1fa': '/content/Megascans Library/Downloaded/UAssets/tlesfb1fa/tlesfb1fa.json', 'uknjfevga': '/content/Megascans Library/Downloaded/UAssets/uknjfevga/uknjfevga.json', 'tk4tcfhfa': '/content/Megascans Library/Downloaded/UAssets/tk4tcfhfa/tk4tcfhfa.json', 'uknkaffaw': '/content/Megascans Library/Downloaded/UAssets/uknkaffaw/uknkaffaw.json', 'extra_test': '/content/Megascans Library/Downloaded/UAssets/extra_test/extra_test.json'}
Assets not yet added to KG: {'extra_test': '/content/Megascans Library/Downloaded/UAssets/extra_test/extra_test.json'}


In [5]:
from rdflib import Namespace
import os
import json
import re

# retrieve asset lists' file. If downloaded asset folder contains assts besides those mentioned in assets list file, update KG

# Set for colab env.
path_to_asset_files = '/content/Megascans Library/UAssets/'

def update_KG(path_to_asset_files):
  # cur_dir = os.chdir(path_to_json_files)
  # # Get list of all asset folder names, then iteratively go through them.
  # assets_path_dict = {}
  
  # asset_folder_names = os.listdir(path_to_asset_files)
  # asset_folder_paths = []
  # for asset_name in asset_folder_names:
  #   asset_folder_path.append(cur_dir+asset_name)
  # print("Updated assets folder path: ", asset_folder_path)
  # for i in range in asset_folder_path:
  #   if asset_
 



  
  n = Namespace("http://www.semanticweb.org/szm/megascan-assets-ontology#")  
  json_files = [pos_json for pos_json in os.listdir(path_to_asset_files) if pos_json.endswith('.json')]

  for asset_file in json_files:

    # Open asset file
    f = open(asset_file)
    data = json.load(f) # json file loaded as dictionary

    # Create asset and set asset name
    name = data["name"]     # obtain name of asset, convert it to RDF format literal
    name_processed = re.sub("[^a-zA-Z0-9]", "_", name)
    print("Created asset name = ", name)
    print(name_processed)
    asset = URIRef(str(n)+"Asset_"+name_processed)  # create a resource in the Assets Knowledge Graph (KG)
    print("Asset = ", asset)
    g.add((asset, RDF.type, n.Asset))
    g.add((asset, n.assetName, Literal(name, datatype=XSD.string)))     # Add triple to KG: {new Asset resource, nameRelation (from schema), name obtained from asset file}

    try:
      # Set asset ID
      id = Literal(str(data["id"]), datatype=XSD.string)
      g.add((asset, n.assetID, id))
      print("Added assetID = ", id)
    except:
      print("No asset ID specified for asset", asset)

    try:
      # Set asset search tags
      tags = data["tags"]
      print("number of tags = ", len(tags))
      for t in tags:
        tag = Literal(str(t), datatype=XSD.string)
        g.add((asset, n.assetTag, tag))
        print("Added tag = ", tag)

    except:
      print("No search tags specified for asset", asset)

    try:
      # Create category individuals and set their names, then assign asset categories
      categories = data["categories"]
      print("number of categories = ", len(categories))
      for c in categories:
        category = Literal(c, datatype=XSD.string)
        g.add((asset, n.assetCategory, category))
        print("Adding category = ", category)

    except:
      print("No categories specified for asset", asset)


    properties = data["properties"]
    print("number of properties = ", len(properties))
    for prop in properties:
      key = list(prop.keys())[0]
      value = list(prop.values())[0]
      print("prop['key'] = "  , prop["key"])
      print("prop['value'] = "  , prop["value"])

    try:
      properties = data["properties"]
      print("number of properties = ", len(properties))
      for prop in properties:
        key = list(prop.keys())[0]
        value = list(prop.values())[0]

        print("prop['key'] = "  , prop["key"])
        print("prop['value'] = "  , prop["value"])

        print("prop key = ", key)
        if prop["key"] == "size":
          # print(value, Literal(value))
          if prop["value"] == "tiny":
            g.add((asset, n.assetSize, n.tiny))
            print("Added asset size = ", n.tiny)
          elif prop["value"] == "small":
            g.add((asset, n.assetSize, n.small))
            print("Added asset size = ", n.small)
          elif prop["value"] == "medium":
            g.add((asset, n.assetSize, n.medium))
            print("Added asset size = ", n.medium)
          elif prop["value"] == "large":
            g.add((asset, n.assetSize, n.large))
            print("Added asset size = ", n.large)
          else:
            g.add((asset, n.assetSize, n.extra_large))
            print("Added asset size = ", n.extra_large)

        elif prop["key"] == "age":
          age = Literal(prop["value"], datatype=XSD.string)
          g.add((asset, n.assetAge, age))
          print("Added asset age = ", age)

    except:
      print("No properties specified for asset", asset)

    try:
      # Set asset average color
      avg_color = Literal(str(data["averageColor"]), datatype=XSD.string)
      g.add((asset, n.assetAvgColor, avg_color))
      print("Added asset average color = ", avg_color)
    except:
      print("No asset avg color specified for asset", asset)




    unique_biomes = set()
    unique_regions = set()

    # Create biome individuals if not alrady created, then link to assets
    try:
      biome = URIRef(str(n)+"Biome_"+data["environment"]["biome"])
      if biome not in unique_biomes:
        g.add((biome, RDF.type, n.Biome))
        biome_name = Literal(data["environment"]["biome"], datatype=XSD.string)
        g.add((biome, n.biomeName, biome_name))
        unique_biomes.add(biome)
        print("Added new biome to graph: ", biome)
      g.add((asset, n.assetEnvironmentBiome, biome))
      print(("Added asset's biome = ", biome))
    except:
      print("No asset biome found for asset", asset)



    # Create region individuals, then link to assets
    try:
      region = URIRef(str(n)+"Region_"+data["environment"]["region"])
      if region not in unique_regions:
        g.add((region, RDF.type, n.Region))
        region_name = Literal(data["environment"]["region"], datatype=XSD.string)
        g.add((region, n.regionName, region_name))    
        unique_regions.add(region)
        print("Added new region to graph: ", region)
      g.add((asset, n.assetEnvironmentRegion, region))
      print(("Added asset's region = ", region))
    except:
      print("No asset region found for asset", asset)

    print(len(g))

    f.close()

  
  g.serialize(destination='Populated_Assets_KG.ttl', format='turtle')


In [ ]:
import os

def trigger_KG_update(path_to_asset_folder)
  cur_dir = os.chdir(path_to_asset_folder)
  # Get list of all asset folder names, then iteratively go through them.
  
  
  downloaded_asset_folder_names = os.listdir(path_to_asset_files)
  downloaded_asset_folder_paths = []
  for asset_name in downloaded_asset_folder_names:
    downloaded_asset_folder_paths.append(cur_dir+asset_name)
  # print("Updated assets folder path: ", downloaded_asset_folder_paths)
  downloaded_assets_path_dict = dict(zip(downloaded_asset_folder_names, downloaded_asset_folder_paths))

# read asset list file. For any asset not mentioned in the assets' list file but present in the downloaded assets' folder, add it to KG
with open('assets_file.txt', assets_file):
  for key,value in downloaded_assets_path_dict:
    
 
